In [25]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
#from scipy.misc import imread, imresize
import imageio as IO
import skimage as sk

In [26]:
def process_image(image):
    img = IO.imread(image)
    img = sk.transform.resize(img, (256, 256))   #256x256x3
    img = img.transpose(2, 0, 1)   #channels first
    img = img / 255.
    img = torch.FloatTensor(img)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    transform = transforms.Compose([normalize])
    img = transform(img)  # (3, 256, 256)
    return img

In [27]:
# Define model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # in_channels = 3, out_channels = 6, filter_size = 5, 0 padding, stride 1
        self.conv1 = nn.Conv2d(3, 6, 5)    
        self.pool = nn.MaxPool2d(2, 2)     
        self.conv2 = nn.Conv2d(6, 12, 5)   
        self.fc1 = nn.Linear(12 * 61 * 61, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        # consider an image of size 256x256
        x = F.relu(self.conv1(x))                # output size = [ (256 - 5 + 2(0) ) / 1 ] + 1 --> 252x252
        x = self.pool(x)                         # output_size = 252 / 2 --> 126x126
        x = F.relu(self.conv2(x))                # output size = [ (126 - 5 + 2(0) ) / 1 ] + 1 --> 122x122
        x = self.pool(x)                         # output size = 122/2 --> 61x61
        x = x.view(-1, 12 * 61 * 61)             # (1, 44652)
        x = F.relu(self.fc1(x))                  # (1, 120)
        x = self.fc2(x)                          # (1, 10)
        return x

In [28]:
# Initialize model
model = CNN()
# Initialize optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [29]:
image = process_image('hymenoptera_data/train/ants/0013035.jpg')
print(image.shape)
image = image.unsqueeze(0)      #batch dimension
print(image.shape)

torch.Size([3, 256, 256])
torch.Size([1, 3, 256, 256])


/tmp/ipykernel_6326/458346200.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = IO.imread(image)


In [30]:
output = model(image)
print(output.shape)

torch.Size([1, 10])


In [31]:
# print model's state dictionary
print("Model's State Dict:")
for param_tensor in model.state_dict():
    print(f"{param_tensor} \t {model.state_dict()[param_tensor].size()}")

Model's State Dict:
conv1.weight 	 torch.Size([6, 3, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([12, 6, 5, 5])
conv2.bias 	 torch.Size([12])
fc1.weight 	 torch.Size([120, 44652])
fc1.bias 	 torch.Size([120])
fc2.weight 	 torch.Size([10, 120])
fc2.bias 	 torch.Size([10])


In [32]:
# Save the model
torch.save(model.state_dict(), 'test_model_save.tar')

In [33]:
# load the saved model and test
# Model still needs to be defined before loading
#model = CNN()
# torch.load -> loads the dictionary into python
# Pass this dictionary to the model with the load_state_dict
model.load_state_dict(torch.load('test_model_save.tar')) # load the saved weights into the current model
model.eval() # To set the model into evaluation mode to ensure that the batch normalization and the drop out work properly. They work differently during training and testing. 

/tmp/ipykernel_6326/4032864886.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('test_model_save.tar')) # load the saved weights into the

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=44652, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
)

In [34]:
# Saving and loading a checkpoint. This when you want to pause and resume training.
checkpoint = {'epoch': 1,  # which epoch we stopped the training at
              'model_state_dict': model.state_dict(), # model state dictionary at the end of the most recent epoch
              'optimizer_state_dict': optimizer.state_dict(), # optimizer at the end of the most recent epoch
              'loss': 0.2} # loss at the end of most recent epoch
torch.save(checkpoint, 'test_model_save.tar')

In [35]:
# model = CNN()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
checkpoint = torch.load('test_model_save.tar')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epochs = checkpoint['epoch']
loss = checkpoint['loss']

/tmp/ipykernel_6326/1572068386.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('test_model_save.tar')


In [39]:
# if testing
model.eval()
# if resuming training 
model.train()

# FYI this is just setting the modes. You need to write the code to actually train or test the model. 

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=44652, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
)